# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load .csv to dataframe
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 

locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Plot map and add heatmap layer
fig = gmaps.figure(center=(38.18,-1.13), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

# Show humidity heatmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Condition #1: Max temperature over 70 degrees but less than 100 degrees
# Condition #2: Humidity over 30% but less than 50%
# Condition #3: Wind speeds less than 20 mph

# Accessing group of values using labels related to conditions set above
weather_criteria = city_data_df.loc[(city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 100) \
                                   & (city_data_df["Humidity"] >= 30) & (city_data_df["Humidity"] <=50) \
                                   & (city_data_df["Wind Speed"] <=20)].dropna()
weather_criteria

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
15,15,albany,40,US,1558378727,45,42.65,-73.75,82.40,12.75
39,39,mayor pablo lagerenza,43,PY,1558378761,33,-19.93,-60.77,91.86,14.90
47,47,port elizabeth,20,US,1558378762,46,39.31,-74.98,93.00,12.75
53,53,cabo san lucas,5,MX,1558378605,34,22.89,-109.91,91.00,11.41
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99
73,73,virginia beach,40,US,1558378597,48,36.85,-75.98,91.00,10.29
75,75,saint george,40,GR,1558378768,33,39.45,22.34,75.20,11.41
76,76,pontes e lacerda,41,BR,1558378768,41,-15.23,-59.33,88.44,3.71
90,90,tarauaca,48,BR,1558378771,47,-8.16,-70.77,90.06,1.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df and return a copy of the set
hotel_df = weather_criteria[["City","Country","Lat","Lng"]].copy()
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,albany,US,42.65,-73.75,
39,mayor pablo lagerenza,PY,-19.93,-60.77,
47,port elizabeth,US,39.31,-74.98,
53,cabo san lucas,MX,22.89,-109.91,
56,pisco,PE,-13.71,-76.20,
68,beja,PT,38.02,-7.87,
73,virginia beach,US,36.85,-75.98,
75,saint george,GR,39.45,22.34,
76,pontes e lacerda,BR,-15.23,-59.33,
90,tarauaca,BR,-8.16,-70.77,


In [7]:
# Build Parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Begin for statement to loop through hotels
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    closest_hotel = requests.get(url, params=params)
    
    closest_hotel = closest_hotel.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = closest_hotel["results"][0]["name"]
    
    # If no record found, skip to next hotel    
    except (KeyError, IndexError):
        print("Hotel not found. Skipping...")
hotel_df

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


,City,Country,Lat,Lng,Hotel Name
15,albany,US,42.65,-73.75,Renaissance Albany Hotel
39,mayor pablo lagerenza,PY,-19.93,-60.77,
47,port elizabeth,US,39.31,-74.98,
53,cabo san lucas,MX,22.89,-109.91,Villa Del Palmar Beach Resort & Spa Cabo San L...
56,pisco,PE,-13.71,-76.20,Embassy Beach
68,beja,PT,38.02,-7.87,Hotel Santa Bárbara
73,virginia beach,US,36.85,-75.98,Hilton Virginia Beach Oceanfront
75,saint george,GR,39.45,22.34,
76,pontes e lacerda,BR,-15.23,-59.33,Plaza Hotel
90,tarauaca,BR,-8.16,-70.77,PRO LINK HOTEL


In [8]:
# Remove rows w/o hotel name
hotel_df_non_empty = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_non_empty.iterrows()]
locations = hotel_df_non_empty[["Lat", "Lng"]]

In [10]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=[f"Hotel Information: {rate}" for rate in hotel_info])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))